In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [3]:
train_dataset = pd.read_pickle('train_forms.pkl')[['input_text','labels']]
train_dataset.drop_duplicates(['input_text']).reset_index(drop=True)
train_dataset['labels'].unique()

array(['cv', 'email', 'firstname', 'lastname', 'submit', 'fake',
       'phonenumber', 'transcript', 'question', 'agree', 'other',
       'address', 'availablefrom', 'salary', 'xing', 'linkedin', 'cancel',
       'picture', 'coverletter', 'housenumber', 'residentcounty',
       'recommendation', 'website', 'workherebefore', 'twitter', 'github',
       'birthdate', 'sex', 'citizencountry', 'postalcode', 'search',
       'findus', 'next', 'cookie', 'dropbox', 'googledrive', 'workhours',
       'visa', 'login', 'eligible', 'residentcountry', 'register', 'city',
       'workduration', 'germanlevel', 'username', 'password', 'company',
       'relocate', 'universityenrollment', 'englishlevel'], dtype=object)

In [4]:
from transformers import Qwen2ForCausalLM
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct'
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Config {
  "_name_or_path": "Qwen/Qwen2-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

In [5]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [6]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj']))
model.print_trainable_parameters()
model

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=896, out_features=128, bias=True)
              (v_proj): lora.Linear(

In [7]:
"""from sentence_transformers  import SentenceTransformer

embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir).to('cuda')
embdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)
embeding_model.eval()
embeding_model"""

"from sentence_transformers  import SentenceTransformer\n\nembeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir).to('cuda')\nembdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)\nembeding_model.eval()\nembeding_model"

In [8]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
c = test_dataset['label']=='phone'
test_dataset[c]

,query,element,label
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone
14,Telephone Number (Mobile/Work)\n,"<input id=""form-field-field_07a6a9e"" name=""for...",phone
22,Phone *\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone
31,Phone\n *\n,"<input aria-required=""true"" autocomplete=""tel""...",phone
36,"Wir haben festgestellt, dass Sie bereits ein K...","<input data-slnm=""applicationform-input_phoneB...",phone
42,E-Mail*\n Bitte E-Mail-Adresse angeben\n,"<input id=""phone"" kameleoonlistener-m6gb=""true...",phone
46,Telefon\n,"<input autocomplete=""tel"" id=""phone"" maxlength...",phone
58,Phone number\n *\n,"<input aria-invalid=""false"" aria-required=""tru...",phone
64,An email is required\n An email is required\n ...,"<input name=""cPhoneNumber"" ng-model=""candidate...",phone
72,handy\n (optional)\n,"<input name=""phone"" placeholder=""+49 123 2222 ...",phone


In [9]:
x = test_dataset['query'] + test_dataset['element']
x = x.tolist()
x[0]

'Personal information\n CV or resume\n Browse CV to autocomplete your application. You can still fill your profile manually.\n <input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/>'

In [10]:
s='['
for i in LABEL_INDEX_TO_KEY[0]:
    s+=f'"{i}",'
    pass

s+=']'
s
#str(LABEL_INDEX_TO_KEY[0])

'["first name","last name","email","password","cv","birth date","phone number","house number","resident country","citizen country","transcript","cover letter","picture","address","linkedin","github","xing","twitter","website","zip","city","salary","sex","available from","captcha","submit","cancel","fake","custom question","privacy","recommend","work here before","find us","dropbox","google drive","work hours","workduration","visa","german language level","company","willing to relocate","university enrollment","english language level","login","username","register","cookie","next","search","other","newsletter",]'

In [11]:
def f(x):
    print(x)
    return '{"answer":"'+ x +'"}'

train_dataset['labels'] = train_dataset['labels'].map(f)
train_dataset


cv
email
firstname
lastname
cv
submit
fake
firstname
lastname
email
phonenumber
transcript
question
agree
agree
agree
agree
submit
other
address
email
firstname
lastname
email
phonenumber
availablefrom
salary
xing
linkedin
agree
agree
submit
cancel
cv
firstname
email
phonenumber
picture
coverletter
question
salary
availablefrom
address
housenumber
address
residentcounty
agree
recommendation
agree
website
agree
agree
agree
workherebefore
workherebefore
agree
submit
firstname
lastname
email
phonenumber
firstname
lastname
email
phonenumber
cv
agree
agree
submit
cv
firstname
email
phonenumber
address
other
linkedin
twitter
github
github
website
agree
firstname
firstname
lastname
birthdate
sex
citizencountry
postalcode
address
address
housenumber
transcript
transcript
salary
xing
linkedin
agree
submit
cancel
submit
linkedin
search
search
cv
submit
findus
availablefrom
linkedin
xing
other
coverletter
question
agree
agree
submit
cv
coverletter
submit
next
other
cancel
other
other
other
other


,input_text,labels
0,personal informationcv or resumebrowse cv to a...,"{""answer"":""cv""}"
1,"<input data-testid=""textinput"" id=""email"" requ...","{""answer"":""email""}"
2,"*<input data-testid=""firstname"" id=""firstname""...","{""answer"":""firstname""}"
3,"*<input data-testid=""lastname"" id=""lastname"" r...","{""answer"":""lastname""}"
4,"documents:.doc, .docx, .pdf, .rtf, .txt<input ...","{""answer"":""cv""}"
...,...,...
596,"upload fileprivacy policy<a href=""https://vert...","{""answer"":""agree""}"
597,"back<a href=""https://bewerber-innendienst.hans...","{""answer"":""cancel""}"
598,"send<a href=""#"" onclick=""javascript: $(this).p...","{""answer"":""submit""}"
599,"input type=""email"" name=""email"" placeholder=""y...","{""answer"":""email""}"


In [12]:

tokenized_data = tokenizer.apply_chat_template([ [#{"role":"system","content":""}
                                                {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                               ,{"role":"user", "content":"What is this HTML element about?\n"+x['input_text'] }
                               ,{"role":"assist","content":x['labels']}] for i,x in train_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

no_label = tokenizer.apply_chat_template([ [#{"role":"system","content":""}
                                                {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                               ,{"role":"user", "content":"What is this HTML element about?\n"+x['element'] }
                               ] for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))

print(tokenizer.decode(token_ids=tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


601
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
["first name","last name","email","password","cv","birth date","phone number","house number","resident country","citizen country","transcript","cover letter","picture","address","linkedin","github","xing","twitter","website","zip","city","salary","sex","available from","captcha","submit","cancel","fake","custom question","privacy","recommend","work here before","find us","dropbox","google drive","work hours","workduration","visa","german language level","company","willing to relocate","university enrollment","english language level","login","username","register","cookie","next","search","other","newsletter",]<|im_end|>
<|im_start|>user
What is this HTML element about?
personal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.<input name="cv" accept="application/

In [13]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
mydc(tokenized_data)

{'input_ids': tensor([[151644,   8948,    198,  ..., 151643, 151643, 151643],
         [151644,   8948,    198,  ..., 151643, 151643, 151643],
         [151644,   8948,    198,  ..., 151643, 151643, 151643],
         ...,
         [151644,   8948,    198,  ..., 151643, 151643, 151643],
         [151644,   8948,    198,  ..., 151643, 151643, 151643],
         [151644,   8948,    198,  ..., 151643, 151643, 151643]]),
 'labels': tensor([[151644,   8948,    198,  ...,   -100,   -100,   -100],
         [151644,   8948,    198,  ...,   -100,   -100,   -100],
         [151644,   8948,    198,  ...,   -100,   -100,   -100],
         ...,
         [151644,   8948,    198,  ...,   -100,   -100,   -100],
         [151644,   8948,    198,  ...,   -100,   -100,   -100],
         [151644,   8948,    198,  ...,   -100,   -100,   -100]])}

In [35]:
trainer =  Trainer(model,
        train_dataset=tokenized_data,
        data_collator=mydc,
        
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=1,
          per_device_train_batch_size=1,
          
            
        ))

trainer.train()

Step,Training Loss
500,0.626200


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=601, training_loss=0.6168772845022293, metrics={'train_runtime': 54.2891, 'train_samples_per_second': 11.07, 'train_steps_per_second': 11.07, 'total_flos': 328648936661760.0, 'train_loss': 0.6168772845022293, 'epoch': 1.0})

In [155]:
model.eval()
i = 14
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                            {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":"What is this HTML element about?\n"+'<input id="password" type="text">'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
print(d)
with torch.no_grad():
    o = model.generate(input_ids = d,max_length = 512)
    print(tokenizer.decode(o[0]))
    


tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,   7771,   4226,    702,    311,
            387,    304,   2951,   3561,   1075,   5212,   9217,  62366,   7810,
           5443,    419,   2213,    304,    697,   4226,    510,   1183,   3896,
            829,   2198,   4259,    829,   2198,   2332,   2198,   3833,   2198,
          13122,   2198,  27713,   2400,   2198,   4844,   1372,   2198,   7675,
           1372,   2198,  39531,   3146,   2198,  53861,  23678,   3146,   2198,
           1458,   1228,   2198,   3688,   6524,   2198,  26483,   2198,   4995,
           2198,  76170,   2198,   5204,   2198,  89551,   2198,  14679,   2198,
          19485,   2198,   9964,   2198,   8926,   2198,  51610,   2198,  11553,
           2198,  10334,    504,   2198,  54347,   2198,   5955,   2198,  18515,
           2198,  30570,   2198,   9163,   3405,   2198,  46821,   2198,  66589,
           2198,   1778,   1

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
["first name","last name","email","password","cv","birth date","phone number","house number","resident country","citizen country","transcript","cover letter","picture","address","linkedin","github","xing","twitter","website","zip","city","salary","sex","available from","captcha","submit","cancel","fake","custom question","privacy","recommend","work here before","find us","dropbox","google drive","work hours","workduration","visa","german language level","company","willing to relocate","university enrollment","english language level","login","username","register","cookie","next","search","other","newsletter",]<|im_end|>
<|im_start|>user
What is this HTML element about?
<input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/><|im_end|>
<|im_start|>assist
{"answer":"cv"}<|im_end|>


In [17]:
$$$

LABEL_INDEX_TO_KEY

SyntaxError: invalid syntax (115010059.py, line 1)

In [ ]:

c = train_dataset['labels']=='relocate'
train_dataset.loc[c,'labels'] = 'willing to relocate'
train_dataset[c]


,input_text,labels


In [ ]:
#train_dataset.to_excel('train_forms.xlsx',index=0)

In [ ]:
$$$
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


SyntaxError: invalid syntax (281372578.py, line 1)

In [ ]:
def batch_tok(example):

    return embdeding_tokenizer(example,padding=True,truncation=True,max_length=512,return_tensors='pt')



LABEL_INDEX_TO_KEY = list(KEYWORDS.keys())

query_token = embdeding_tokenizer(df_forms_1['query'].tolist(),padding=False,truncation=True,max_length=512)
element_token = embdeding_tokenizer(df_forms_1['element'].tolist(),padding=False,truncation=True,max_length=512)
classes_token = embdeding_tokenizer(LABEL_INDEX_TO_KEY,padding=True,truncation=True,max_length=512,return_tensors='pt')


def myheatmap(x):
    map =[]
    for i in range(len(x)):
        sim = torch.cosine_similarity(x,x[i])
        map.append(sim)
    
    
    plt.figure(figsize=(10,10))
    sns.heatmap(map)
    plt.show()
    
    

with torch.no_grad():
    
    output = embeding_model(**classes_token)
    #class_embdeding = torch.mean(output.last_hidden_state,1)
    class_embdeding = mean_pooling(output,classes_token['attention_mask'])
    class_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    #sim = torch.cosine_similarity(class_embdeding,class_embdeding[0])
    
    #print(sim.shape)
    pass
    
i = 151
with torch.no_grad():
    x = torch.tensor(element_token['input_ids'][i:i+1])
    #print(x)
    output = embeding_model(input_ids= x)
    #query_embdeding = torch.mean(output.last_hidden_state,1)
    query_embdeding = mean_pooling(output,torch.tensor(element_token['attention_mask'][i:i+1]))
    #query_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    
    sim = torch.cosine_similarity(class_embdeding,query_embdeding)
    print(pd.DataFrame({'value':sim,'label':LABEL_INDEX_TO_KEY}).sort_values('value',ascending=False)[:5])
    print(LABEL_INDEX_TO_KEY[ sim.argmax() ], embdeding_tokenizer.decode(token_ids=x[0]))
    print(df_forms_1['element'][i] )
    

#myheatmap(class_embdeding)
